In [14]:
# import comet_ml at the top of your file

import requests
import subprocess


In [25]:

# img_data = requests.get("https://d1iczm3wxxz9zd.cloudfront.net/971e55cc-128e-4809-bb05-2adf7b0238ec/000000-0000000000/39127110424036069919666352160427857710300904072429089980813045768686488932997/ITEM_PREVIEW1.jpg").content
# with open('pineapple.jpg', 'wb') as handler:
#     handler.write(img_data)



def make_grid(possible_values):
    possible_values = possible_values.items()
    result = [{}]
    for k, vals in possible_values:
        result = [dict(**x, **{k: v}) for x in result for v in vals]
    return result


def get_git_commit():
    return (
        subprocess.check_output("git rev-parse HEAD", shell=True)
        .decode("utf-8")
        .strip()
    )

def params_to_string(params):
    return '\n'.join([f"{k}={v}" for k, v in params.items()])




def hyperparameter_search(possible_values):
    git_commit = get_git_commit()

    api_url = 'https://jobs.redwoodresearchcompute.com:10101/api'
    json_dict = {
        'token': 'tao-1637086550-0-0a7d7030fbb8e316da58c6afce7c6315',
        'git_commit': git_commit,
        'git_repo': 'redwoodresearch/mlab',
        'git_branch': 'warren',
        'kind': 'submitJob',
        'command': ['python', 'days/w1d4/run_experiment.py'],
        'name': 'warren lukas parameter search',
        'tasks': [{
            'priority': 1,
            'parameters': {
                'gin_config': params_to_string(params)
            }
        } for params in make_grid(possible_values)],
        'scheduling_requirements': {'schedulability': True, 'resource_options': [["A100"], ["RTX3090"], ["V100"]]},

    }
    response = requests.post(api_url, json = json_dict)
    print(response.content)

# if __name__ == 'main':

possible_values = {
    'train.lr': [0.1, 0.01, 0.001],
    'train.momentum': [0, 0.4, 0.8],
    'train.epochs': [1,2]
}
hyperparameter_search(possible_values)

# hyperparameter_search(possible_values, train)

b'{"job_id": 419, "task_ids": [3985, 3986, 3987, 3988, 3989, 3990, 3991, 3992, 3993, 3994, 3995, 3996, 3997, 3998, 3999, 4000, 4001, 4002]}'


In [22]:
'\n'.join(['a', 'b'])

'a\nb'